In [ ]:
import os
import cv2
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras import optimizers, applications
from keras.applications import ResNet50
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

seed = 0
%matplotlib inline
sns.set(style='whitegrid')
warnings.filterwarnings('ignore')

In [ ]:
train_15 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv')
test_15 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/testLabels15.csv')

train_19 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/trainLabels19.csv')
test_19 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/testImages19.csv')

In [ ]:
#removendo a coluna Usage
test_15.drop('Usage',axis=1, inplace=True)

In [ ]:
#renomeando as colunas de 2015
train_15.columns = train_19.columns
test_15.columns = train_19.columns

In [ ]:
train_15['id_code'] = train_15['id_code'].apply(lambda x:'../input/resized-2015-2019-blindness-detection-images/resized train 15/'+x+'.jpg')
test_15['id_code'] = test_15['id_code'].apply(lambda x:'../input/resized-2015-2019-blindness-detection-images/resized test 15/'+x+'.jpg')
train_19['id_code'] = train_19['id_code'].apply(lambda x:'../input/resized-2015-2019-blindness-detection-images/resized train 19/'+x+'.jpg')
test_19['id_code'] = test_19['id_code'].apply(lambda x:'../input/resized-2015-2019-blindness-detection-images/resized test 19/'+x+'.jpg')

In [ ]:
#concatenar os dados de train e test 2015
old_data = train_15.append(test_15)
old_data['data'] = 'old'

In [ ]:
old_data['diagnosis'].value_counts()

### Reduzir em 75% os diagnosis iguais a 0, para balancear os dados de 2015

In [ ]:
diag_0 = old_data[old_data['diagnosis'] == 0].sample(frac=0.25,random_state=seed)
diag_dif_0 = old_data[old_data['diagnosis'] != 0]
old_data = diag_0.append(diag_dif_0)

In [ ]:
train_19['data'] = 'new'

### Concatenar old_data com train_19

In [ ]:
train = train_19.append(old_data)
print('Number of train samples: ', train.shape[0])
print('Number of test samples: ', test_19.shape[0])
display(train.head())

In [ ]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for fold_n, (trn_idx, val_idx) in enumerate(folds.split(train_19, train_19['diagnosis'])):
    print('Fold: %s, Train size: %s, Validation size %s' % 
          (fold_n, len(trn_idx), len(val_idx)))
    train_19[('fold_%s' % fold_n)] = 0
    train_19[('fold_%s' % fold_n)].loc[trn_idx] = 'train'
    train_19[('fold_%s' % fold_n)].loc[val_idx] = 'validation'
    old_data[('fold_%s' % fold_n)] = 0
    old_data[('fold_%s' % fold_n)] = 'train'
    
train_df = old_data.append(train_19)
display(train_df.head())

### Train and validation label distribution

In [ ]:
for fold_n, (trn_idx, val_idx) in enumerate(folds.split(train_df, train_df['diagnosis'])):
    fig, (ax1, ax2) = plt.subplots(1, 2, sharex='col', figsize=(24, 6))
    fig.suptitle('Fold%s'%fold_n, fontsize=22)    
    sns.countplot(x="diagnosis", data=train_df[train_df[('fold_%s' % fold_n)] == 'train'], palette="GnBu_d", ax=ax1).set_title('Train')
    sns.countplot(x="diagnosis", data=train_df[train_df[('fold_%s' % fold_n)] == 'validation'], palette="GnBu_d", ax=ax2).set_title('Validation')
    sns.despine()
    plt.show()

### save 5-fold

In [ ]:
train_df.to_csv('5-fold.csv', index=False)
print('Number of train samples: ', train_df[train_df[('fold_%s' % fold_n)] == 'train'].shape[0])
print('Number of validation samples: ', train_df[train_df[('fold_%s' % fold_n)] == 'validation'].shape[0])
display(train_df.head())